In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sarlren/isic16")

print("Path to dataset files:", path)

100%|██████████| 832M/832M [00:39<00:00, 22.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/sarlren/isic16/versions/2


In [2]:
!git clone https://github.com/WuJunde/MedSegDiff.git
!mv MedSegDiff/* ./

Cloning into 'MedSegDiff'...
remote: Enumerating objects: 716, done.
remote: Counting objects: 100% (397/397), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 716 (delta 340), reused 248 (delta 248), pack-reused 319 (from 2)
Receiving objects: 100% (716/716), 3.90 MiB | 28.16 MiB/s, done.
Resolving deltas: 100% (396/396), done.


In [3]:
!mkdir /kaggle

mkdir: cannot create directory ‘/kaggle’: File exists


In [4]:
!mkdir /kaggle/working

In [5]:
!mkdir /kaggle/working/out

In [6]:
!mkdir /kaggle/working/isic

In [7]:
!cp -r /content/guided_diffusion /content/scripts/guided_diffusion

In [8]:
import subprocess
subprocess.call("cp -r "+path+"/ISBI2016_ISIC_Part1_Test_Data/* /kaggle/working/isic/", shell=True)
subprocess.call("cp -r "+path+"/ISBI2016_ISIC_Part1_Training_Data/* /kaggle/working/isic/", shell=True)
subprocess.call("cp -r "+path+"/ISBI2016_ISIC_Part1_Training_GroundTruth/* /kaggle/working/isic/", shell=True)
subprocess.call("cp -r "+path+"/ISBI2016_ISIC_Part1_Test_GroundTruth/* /kaggle/working/isic/", shell=True)
subprocess.call("cp "+path+"/ISBI2016_ISIC_Part3B_Test_GroundTruth.csv /kaggle/working/isic/", shell=True)
subprocess.call("cp "+path+"/ISBI2016_ISIC_Part3B_Training_GroundTruth.csv /kaggle/working/isic/", shell=True)

0

In [9]:
!pip install blobfile
!pip install nibabel
!pip install batchgenerators
!pip install visdom
!pip install torchsummary
!pip install mpi4py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 10.3 MB/s eta 0:00:00
  Created wheel for batchgenerators: filename=batchgenerators-0.25.1-py3-none-any.whl size=93088 sha256=adf8ed0a3c39e2dbe8a434f4a0452c614bbaed156cc7ec1222d033f5faca43f6
  Stored in directory: /root/.cache/pip/wheels/56/11/c7/fadca30e054c602093ffe36ba8a2f0a87dd2f86ac75191d3ed
Successfully built batchgenerators


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 71.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408195 sha256=b4a753595fa3303267f1fa6c24160a2ebfbc8ee0ac64e9d7faaecceabe1a1feb
  Stored in directory: /root/.cache/pip/wheels/fa/a4/bb/2be445c295d88a74f9c0a4232f04860ca489a5c7c57eb959d9
Successfully built visdom
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.3/466.3 kB 31.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.3-cp311-cp311-linux_x86_64.whl size=4458267 sha256=b0f3cbc3ebcd632f3fe5fe80be88bf204220acaa8d1dbd9d9e7f704b573d7080
  Stored in directory: /root/.cache/pip/wheels/5c/56/17/bf6ba37aa971a191a8b9eaa188bf5ec855b8911c1c56fb1f84
Successfully built mpi4py


In [21]:
import sys
import argparse
sys.path.append("/content/scripts")
sys.path.append("./")
from guided_diffusion import dist_util, logger
from guided_diffusion.resample import create_named_schedule_sampler
from guided_diffusion.bratsloader import BRATSDataset, BRATSDataset3D
from guided_diffusion.isicloader import ISICDataset
from guided_diffusion.custom_dataset_loader import CustomDataset
from guided_diffusion.script_util import (
    model_and_diffusion_defaults,
    create_model_and_diffusion,
    args_to_dict,
    add_dict_to_argparser,
)
import torch as th
from guided_diffusion.train_util import TrainLoop
from visdom import Visdom
viz = Visdom(port=8850)
import torchvision.transforms as transforms

data_name = 'ISIC'
data_dir = "/kaggle/working/isic"
out_dir = "/kaggle/working/out"
image_size = 256
num_channels = 128
class_cond = False
num_res_blocks = 2
num_heads = 1
learn_sigma = True
use_scale_shift_norm = False
attention_resolutions = 16
diffusion_steps = 1000
noise_schedule = 'linear'
rescale_learned_sigmas = False
rescale_timesteps = False
lr = 1e-4
batch_size = 4
patience = 2
min_delta = 0.001
microbatch = -1  # -1 disables microbatches
ema_rate = "0.9999"  # comma-separated list of EMA values
log_interval = 100
save_interval = 5000
resume_checkpoint = None #"/results/pretrainedmodel.pt"

def main():
    #args = create_argparser().parse_args()

    #dist_util.setup_dist(args)
    logger.configure(dir = out_dir)

    logger.log("creating data loader...")

    if data_name == 'ISIC':
        tran_list = [transforms.Resize((image_size,image_size)), transforms.ToTensor(),]
        transform_train = transforms.Compose(tran_list)

        ds = ISICDataset(data_dir, transform_train)
        in_ch = 4
    elif data_name == 'BRATS':
        tran_list = [transforms.Resize((image_size, image_size)),]
        transform_train = transforms.Compose(tran_list)

        ds = BRATSDataset3D(data_dir, transform_train, test_flag=False)
        in_ch = 5
    else :
        tran_list = [transforms.Resize((image_size, image_size)), transforms.ToTensor(),]
        transform_train = transforms.Compose(tran_list)
        print("Your current directory : ", data_dir)
        ds = CustomDataset(data_dir, transform_train)
        in_ch = 4

    datal= th.utils.data.DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=True)
    data = iter(datal)

    logger.log("creating model and diffusion...")

    model, diffusion = create_model_and_diffusion(
        **args_to_dict(model_and_diffusion_defaults().keys())
    )
    #if multi_gpu:
    #    model = th.nn.DataParallel(model,device_ids=[int(id) for id in multi_gpu.split(',')])
    #    model.to(device = th.device('cuda', int(gpu_dev)))
    #else:
    #    model.to(dist_util.dev())
    schedule_sampler = create_named_schedule_sampler(schedule_sampler, diffusion,  maxt=diffusion_steps)


    logger.log("training...")
    TrainLoop(
        model=model,
        diffusion=diffusion,
        classifier=None,
        data=data,
        dataloader=datal,
        batch_size=batch_size,
        microbatch=microbatch,
        lr=lr,
        ema_rate = ema_rate,
        log_interval = log_interval,
        save_interval=save_interval,
        resume_checkpoint=resume_checkpoint,
        use_fp16=use_fp16,
        fp16_scale_growth=fp16_scale_growth,
        schedule_sampler=schedule_sampler,
        weight_decay = weight_decay,
        lr_anneal_steps =lr_anneal_steps,
        patience =patience,  # Add early stopping patience parameter
        min_delta = min_delta,  # Add minimum improvement threshold
    ).run_loop()


def create_argparser():
    defaults = dict(
        data_name = 'BRATS',
        data_dir= "/kaggle/working/isic/",
        schedule_sampler="uniform",
        lr=1e-4,
        weight_decay=0.0,
        lr_anneal_steps=0,
        batch_size=1,
        microbatch=-1,  # -1 disables microbatches
        ema_rate="0.9999",  # comma-separated list of EMA values
        log_interval=100,
        save_interval=5000,
        resume_checkpoint=None, #"/results/pretrainedmodel.pt"
        use_fp16=False,
        fp16_scale_growth=1e-3,
        gpu_dev = "0",
        multi_gpu = None, #"0,1,2"
        out_dir='./results/'
    )
    defaults.update(model_and_diffusion_defaults())
    parser = argparse.ArgumentParser()
    add_dict_to_argparser(parser, defaults)
    return parser


if __name__ == "__main__":
    main()

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 493, in _make_request
    conn.request(
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 445, in reque

Exception in user code:
------------------------------------------------------------


ERROR:visdom:[Errno 99] Cannot assign requested address
ERROR:websocket:[Errno 99] Cannot assign requested address - goodbye
ERROR:websocket:error from callback <function Visdom.setup_socket.<locals>.on_close at 0x795aa556a160>: Visdom.setup_socket.<locals>.on_close() takes 1 positional argument but 3 were given
ERROR:visdom:Visdom.setup_socket.<locals>.on_close() takes 1 positional argument but 3 were given
ERROR:visdom:[Errno 99] Cannot assign requested address
ERROR:websocket:[Errno 99] Cannot assign requested address - goodbye
ERROR:websocket:error from callback <function Visdom.setup_socket.<locals>.on_close at 0x795aa556bb00>: Visdom.setup_socket.<locals>.on_close() takes 1 positional argument but 3 were given
ERROR:visdom:Visdom.setup_socket.<locals>.on_close() takes 1 positional argument but 3 were given
ERROR:visdom:[Errno 99] Cannot assign requested address
ERROR:websocket:[Errno 99] Cannot assign requested address - goodbye
ERROR:websocket:error from callback <function Visdo

Logging to /kaggle/working/out
creating data loader...


TypeError: expected str, bytes or os.PathLike object, not Compose

OLD

In [10]:
!python3 scripts/segmentation_train.py --data_name ISIC --data_dir /kaggle/working/isic --out_dir /kaggle/working/out --image_size 256 --num_channels 128 --class_cond False --num_res_blocks 2 --num_heads 1 --learn_sigma True --use_scale_shift_norm False --attention_resolutions 16 --diffusion_steps 1000 --noise_schedule linear --rescale_learned_sigmas False --rescale_timesteps False --lr 1e-4 --batch_size 4

Setting up a new session...
Exception in user code:
------------------------------------------------------------
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 493, in _make_req

In [11]:
!ls /kaggle/working/out

emasavedmodel_0.9999_000000.pt	optsavedmodel000000.pt	savedmodel000000.pt
log.txt				progress.csv


In [12]:
%cd /kaggle/working/out
from IPython.display import FileLink
FileLink('/kaggle/working/out/optsavedmodel000000.pt')

/kaggle/working/out


/kaggle/working/out/optsavedmodel000000.pt

In [13]:
!tar -cf models.tar /kaggle/working/out

tar: Removing leading `/' from member names
tar: /kaggle/working/out/models.tar: file is the archive; not dumped


In [ ]:
!ls /kaggle/working/out

In [ ]:
!tar czvf model_tar2.tar.gz /kaggle/working/out

In [ ]:
!ls -alh

In [ ]:
!tar -tvf ./model_tar2.tar.gz

In [ ]:
!nvidia_smi

In [ ]:
!python3 /content/scripts/segmentation_sample.py --data_name ISIC --data_dir /kaggle/working/isic --out_dir /kaggle/working/out --model_path /kaggle/working/out/savedmodel000000.pt --image_size 256 --num_channels 128 --class_cond False --num_res_blocks 2 --num_heads 1 --learn_sigma True --use_scale_shift_norm False --attention_resolutions 16 --diffusion_steps 1000 --noise_schedule linear --rescale_learned_sigmas False --rescale_timesteps False --num_ensemble 5

In [ ]:
!ls /kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/

In [ ]:
!ls /kaggle/working/out
from matplotlib import pyplot as plt
from PIL import Image
display(Image.open('/kaggle/working/out/0010016_output_ens.jpg'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/ISIC_0010016_Segmentation.png'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_Data/ISIC_0010016.jpg'))

display(Image.open('/kaggle/working/out/0010336_output_ens.jpg'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/ISIC_0010336_Segmentation.png'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_Data/ISIC_0010336.jpg'))

display(Image.open('/kaggle/working/out/0010437_output_ens.jpg'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/ISIC_0010437_Segmentation.png'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_Data/ISIC_0010437.jpg'))

display(Image.open('/kaggle/working/out/0010494_output_ens.jpg'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/ISIC_0010494_Segmentation.png'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_Data/ISIC_0010494.jpg'))

display(Image.open('/kaggle/working/out/0010574_output_ens.jpg'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_GroundTruth/ISIC_0010574_Segmentation.png'))
display(Image.open('/kaggle/working/isic/ISBI2016_ISIC_Part1_Test_Data/ISIC_0010574.jpg'))